## Overview

This demo is a simple example of how to use CVIs in batch mode.
Here, we load a simple dataset and run a basic clustering algorithm to prescribe a set of clusters to the features.
It is a combination of these features and the prescribed labels that are used to compute the criterion value.
This simple example demonstrates the usage of a single CVI, but it may be substituted for any other CVI in the `ClusterValidityIndices.jl` package.

## Clustering

### Data Setup

First, we must load all of our dependencies.
We will load the `ClusterValidityIndices.jl` along with some data utilities and the Julia `Clustering.jl` package to cluster that data.

In [1]:
using ClusterValidityIndices    # CVI/ICVI
using Clustering                # DBSCAN
using MLDatasets                # Iris dataset
using MLDataUtils               # Shuffling and splitting
using Printf                    # Formatted number printing

We will download the Iris dataset for its small size and benchmark use for clustering algorithms.

In [2]:
Iris.download(i_accept_the_terms_of_use=true)
features, labels = Iris.features(), Iris.labels()

([5.1 4.9 … 6.2 5.9; 3.5 3.0 … 3.4 3.0; 1.4 1.4 … 5.4 5.1; 0.2 0.2 … 2.3 1.8], ["Iris-setosa", "Iris-setosa", "Iris-setosa", "Iris-setosa", "Iris-setosa", "Iris-setosa", "Iris-setosa", "Iris-setosa", "Iris-setosa", "Iris-setosa"  …  "Iris-virginica", "Iris-virginica", "Iris-virginica", "Iris-virginica", "Iris-virginica", "Iris-virginica", "Iris-virginica", "Iris-virginica", "Iris-virginica", "Iris-virginica"])

Because the MLDatasets package gives us Iris labels as strings, we will use the `MLDataUtils.convertlabel` method with the `MLLabelUtils.LabelEnc.Indices` type to get a list of integers representing each class:

In [3]:
labels = convertlabel(LabelEnc.Indices{Int}, labels)
unique(labels)

3-element Array{Int64,1}:
 1
 2
 3

### Fuzzy C-Means

Get the Fuzzy C-Means clustering result

In [4]:
results = fuzzy_cmeans(features, 3, 2)

FuzzyCMeansResult: 3 clusters for 150 points in 4 dimensions (converged in 18 iterations)

Because the results are fuzzy weights, find the maximum elements along each sample

In [5]:
indices = argmax(results.weights, dims=2)

150×1 Array{CartesianIndex{2},2}:
 CartesianIndex(1, 3)
 CartesianIndex(2, 3)
 CartesianIndex(3, 3)
 CartesianIndex(4, 3)
 CartesianIndex(5, 3)
 CartesianIndex(6, 3)
 CartesianIndex(7, 3)
 CartesianIndex(8, 3)
 CartesianIndex(9, 3)
 CartesianIndex(10, 3)
 ⋮
 CartesianIndex(142, 1)
 CartesianIndex(143, 2)
 CartesianIndex(144, 1)
 CartesianIndex(145, 1)
 CartesianIndex(146, 1)
 CartesianIndex(147, 2)
 CartesianIndex(148, 1)
 CartesianIndex(149, 1)
 CartesianIndex(150, 2)

Get those labels as a vector of integers

In [6]:
c_labels = vec([c[2] for c in indices])

150-element Array{Int64,1}:
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 ⋮
 1
 2
 1
 1
 1
 2
 1
 1
 2

## CVI Criterion Value Extraction

Now that we have some data and a clustering algorithm's prescribed labels, we can compute a criterion value using a CVI in batch mode.
First, we create a CVI object with the default constructor:

In [7]:
# Create a CVI object
my_cvi = CH()

CH(Dict{Int64,Int64}(), 0, 0, Float64[], Integer[], Array{Float64}(undef,0,0), Float64[], Float64[], Array{Float64}(undef,0,0), 0.0, 0.0, 0, 0.0)

Finally we can simply get the criterion value in batch by passing all of the data and Fuzzy C-Means labels at once.

In [8]:
# Get the batch criterion value
criterion_value = get_cvi!(my_cvi, features, c_labels)

558.9999681899934

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*